In [0]:
!unzip lstm-生成cpp代码.zip

In [3]:
!ls

assets		lstm_generate_cpp_code.ipynb  sample_data
generate	lstm-生成cpp代码.zip	      splitCppData.py
getCppFiles.py	__pycache__		      tempCodeRunnerFile.py
helper.py	randomSequence.py	      trained_rnn_1.pt
imgs		readme.md


In [4]:
from splitCppData import splitData

char2int, int2char, int_text, max_int = splitData()
print(len(int_text), max_int)

2691 9543


In [5]:
maxx = 0
for each in int_text:
    for any in each:
        maxx = max(maxx, any)

print(maxx)

9543


In [6]:
from randomSequence import getTrainLoader

train_loader = getTrainLoader()

766502
766502


In [7]:
max(max(int_text))

7181

In [8]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')
else:
    print('GPU is prepared!')

GPU is prepared!


In [0]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim)
        # 之前是vocab_size而不是vocab_size + 1. 一直报错: RuntimeError: index out of range at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/TH/gene.
        # 参考: https://blog.csdn.net/Geek_of_CSDN/article/details/86527107
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                           dropout=dropout, batch_first=True)
        
        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        x = nn_input
        
        batch_size = x.size(0)
        
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # sig_out = self.sig(out)
        # 根据交流群同学的提示, 这里要取消sigmoid. 记得之前做卷积神经网络项目的时候也出现过这个问题
        sig_out = out
        
        sig_out = sig_out.view(batch_size, -1, self.output_size)
        sig_out = sig_out[:, -1]
        
        # return one batch of output word scores and the hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if train_on_gpu:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                     weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

# """
# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
# """
# tests.test_rnn(RNN, train_on_gpu)

### 定义前向及后向传播

通过你实现的 RNN 类来进行前向及后项传播。你可以在训练循环中，不断地调用如下代码来实现：
```
loss, hidden = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

函数中需要返回一个批次以及其隐藏状态的loss均值，你可以调用一个函数`RNN(inp, hidden)`来实现。记得，你可以通过调用`loss.item()` 来计算得到该loss。

**如果使用 GPU，你需要将你的数据存到 GPU 的设备上。**

In [0]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
    
    # perform backpropagation and optimization
    rnn.zero_grad()
    # 修改
    hidden = tuple([each.data for each in hidden])
    output, hidden = rnn(inp, hidden)
    # 修改
    loss = criterion(output, target.long())
    loss.backward()
    
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
# """
# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
# """
# tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

## 神经网络训练

神经网络结构完成以及数据准备完后，我们可以开始训练网络了。

### 训练循环

训练循环是通过 `train_decoder` 函数实现的。该函数将进行 epochs 次数的训练。模型的训练成果会在一定批次的训练后，被打印出来。这个“一定批次”可以通过`show_every_n_batches` 来设置。你会在下一节设置这个参数。

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                if batch_i % 100 == 0:
                    helper.save_model('./trained_rnn' + str(batch_i), rnn)
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### 超参数

设置并训练以下超参数:
-  `sequence_length`，序列长度 
-  `batch_size`，分批大小
-  `num_epochs`，循环次数
-  `learning_rate`，Adam优化器的学习率
-  `vocab_size`，唯一标示词汇的数量
-  `output_size`，模型输出的大小 
-  `embedding_dim`，词嵌入的维度，小于 vocab_size
-  `hidden_dim`， 隐藏层维度
-  `n_layers`， RNN的层数
-  `show_every_n_batches`，打印结果的频次

如果模型没有获得你预期的结果，调整 `RNN`类中的上述参数。

In [0]:
# Data params
# Sequence Length
sequence_length = 20  # of words in a sequence
# Batch Size
batch_size = 128

In [0]:
# Training parameters
# Number of Epochs
num_epochs = 1
# Learning Rate
learning_rate = 0.0018

# Model parameters
# Vocab size
vocab_size = max_int + 1  # 词的数量, 从0开始索引, 所以需要+1
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 256
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 10

### 训练
下一节，通过预处理数据来训练神经网络。如果你的loss结果不好，可以通过调整超参数来修正。通常情况下，大的隐藏层及层数会带来比较好的效果，但同时也会消耗较长的时间来训练。
> **你应该努力得到一个低于3.5的loss** 

你也可以试试不同的序列长度，该参数表明模型学习的范围大小。

In [0]:
import numpy as np
import helper

In [0]:
def run():
    """
    DON'T MODIFY ANYTHING IN THIS CELL
    """

    # create model and move to gpu if available
    rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.0)
    if train_on_gpu:
        rnn.cuda()

    # defining loss and optimization functions for training
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # training the model
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

    # saving the trained model
    helper.save_model('./trained_rnn', trained_rnn)
    print('Model Trained and Saved')

In [0]:
run()

Training for 1 epoch(s)...


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch:    1/1     Loss: 6.724790143966675

Epoch:    1/1     Loss: 5.373217916488647

Epoch:    1/1     Loss: 5.0091150283813475

Epoch:    1/1     Loss: 4.968357992172241

Epoch:    1/1     Loss: 4.847093152999878

Epoch:    1/1     Loss: 4.915909004211426

Epoch:    1/1     Loss: 4.835130214691162

Epoch:    1/1     Loss: 4.8013081550598145

Epoch:    1/1     Loss: 4.773309421539307

Epoch:    1/1     Loss: 4.760389804840088

Epoch:    1/1     Loss: 4.714656972885132

Epoch:    1/1     Loss: 4.504742097854614

Epoch:    1/1     Loss: 4.436546444892883

Epoch:    1/1     Loss: 4.294315767288208

Epoch:    1/1     Loss: 4.107592248916626

Epoch:    1/1     Loss: 4.161919784545899

Epoch:    1/1     Loss: 3.8466867208480835

Epoch:    1/1     Loss: 3.6953810930252073

Epoch:    1/1     Loss: 3.6682599067687987

Epoch:    1/1     Loss: 3.474245381355286

Epoch:    1/1     Loss: 3.23008074760437

Epoch:    1/1     Loss: 3.2118019819259644

Epoch:    1/1     Loss: 3.2235686779022217

Epoch

In [20]:
import torch
torch.__version__

'1.2.0'

In [43]:
import torch
import helper
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    pad_value = 0 # 这个设置成0好些, 其他好像都有点坑
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-len(prime_id):] = np.array(prime_id)
    predicted = [int_to_vocab[each] for each in prime_id]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
          current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq.cpu(), -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

gen_length = 400 # modify the length to your preference
prime_word = ['#', 'include', '<', 'iostream'] # name for starting the script

trained_rnn = helper.load_model('./trained_rnn_1')
generated_script = generate(trained_rnn, [char2int[each] for each in prime_word], int2char, gen_length)
print(generated_script)

# include < iostream > # include < vector > using namespace std ; int main () { int a [ 1001 ] , n , i = 0 ; for (register int i = 0 ; i < n ; + + i ) { cout < < n [ i ] ; cout < < endl ; } int main () { int n , m , m ; cin > > n ; for (int i = 1 ; i < = n ; + + i ) { cin > > a > > b ; for (int i = 1 ; i < = 4 ; i + + ) { if (! vis [ v ] [ i ] & & s [ j + + ] = = 0 ) return false ; } if ((i + 1 ) % 2 = 1 ; if (n < = m ) { if (i = = 0 ) { tmp + = intToStr (f _ m , i , x , y , y ) ; } } } void loop (int x ) { for (int i = 1 ; i < = 10 ; i + + ) { for (int j = 1 ; j < = n ; + + l ) { / / } cout < < " No \ n \ n " ; / / cout < < " - - - - - - - - - - - - - - - - - - - - - - " < < endl ; cout < < endl ; } int main () { int n ; while (scanf (" % d " , & a [ i ] ) ; for (int j = 0 ; j < 3 ; j + + ) { / / cout < < endl ; } / * 3 . 0 , 0 , 0 . 0 , 0 , 0 ) ; for (int i = 1 ; i < = m ; + + i ) { if (str [ i ] [ j ] = ' 0 ' ; } else { if (str [ i ] = = ' = ' ) return true ;


### 问题: 你如何决定你的模型超参数？
比如，你是否试过不同的 different sequence_lengths 并发现哪个使得模型的收敛速度变化？那你的隐藏层数和层数呢？你是如何决定使用这个网络参数的？

**答案:** 

我调整了各种参数, 比如sequence_lengths, 刚开始设置成100, 但发现效果不是很好. 后来又设置成20, 效果较好. 

我的hidden_dim设置成了512, embedding_dim设置成了256, n_layers设置为2, 这几个参数参数越大, 意味着模型越复杂, 在经过很多次调整和测试之后, 我发现这几个参数的设置是相对来说更合理的.

我训练了7个epoch, 最终的loss值为3.25左右.

---



*italicized text*---
# 检查点

通过运行上面的训练单元，你的模型已经以`trained_rnn`名字存储，如果你存储了你的notebook， **你可以在之后的任何时间来访问你的代码和结果**. 下述代码可以帮助你重载你的结果!

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## 生成电视剧剧本
你现在可以生成你的“假”电视剧剧本啦！

### 生成文字
你的神经网络会不断重复生成一个单词，直到生成满足你要求长度的剧本。使用 `generate` 函数来完成上述操作。首先，使用 `prime_id` 来生成word id，之后确定生成文本长度 `predict_len`。同时， topk 采样来引入文字选择的随机性!

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
          current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq.cpu(), -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

gen_length = 500 # modify the length to your preference
prime_word = 'int' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

### 生成一个新剧本
是时候生成一个剧本啦。设置`gen_length` 剧本长度，设置 `prime_word`为以下任意词来开始生成吧:
- "jerry"
- "elaine"
- "george"
- "kramer"

你可以把prime word 设置成 _任意 _ 单词, 但是使用名字开始会比较好(任何其他名字也是可以哒!)

In [0]:
# run the cell multiple times to get different results!
gen_length = 500 # modify the length to your preference
prime_word = 'int' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry:.

elaine: what is this? i thought you liked her?

george: no.

jerry:(smiling) i don't want to get out of here.

george:(smiling, not) oh, no!

george: i don't know what to do, you know, it's a good idea.

jerry: you don't know how you feel?

elaine:(quietly).(to kramer)

kramer:(quietly)

jerry:(on the phone) yeah, that's what they do.

kramer: oh, yeah.

elaine: you got that?

jerry:(looking up to the van) hey, you got it?

jerry:(to the manager of the conversation) oh, you don't have to do that.

elaine: you have to go to the bathroom.(george shakes his head)

elaine: oh! i think you should see that.

george:(to the waitress) yeah.

jerry:(to kramer) hey, you know what? i was just trying to get away, i know what i'm saying.

kramer: i don't have to...

george:(quietly) oh, you got it.

jerry: oh, hi.

jerry: hi.

george:(to jerry) well, i don't know.

jerry:(to george) i know... i know...

kramer: well, i don't want to know if i can do something.

george: what do you think?



#### 存下你最爱的片段

一旦你发现一段有趣或者好玩的片段，就把它存下啦！

In [0]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# 这个电视剧剧本是无意义的
如果你的电视剧剧本不是很有逻辑也是ok的。下面是一个例子。

### 生成剧本案例

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.


如果这个电视剧剧本毫无意义，那也没有关系。我们的训练文本不到一兆字节。为了获得更好的结果，你需要使用更小的词汇范围或是更多数据。幸运的是，我们的确拥有更多数据！在本项目开始之初我们也曾提过，这是[另一个数据集](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data)的子集。我们并没有让你基于所有数据进行训练，因为这将耗费大量时间。然而，你可以随意使用这些数据训练你的神经网络。当然，是在完成本项目之后。
# 提交项目
在提交项目时，请确保你在保存 notebook 前运行了所有的单元格代码。请将 notebook 文件保存为 "dlnd_tv_script_generation.ipynb"，并将它作为 HTML 文件保存在 "File" -> "Download as" 中。请将 "helper.py" 和 "problem_unittests.py" 文件一并打包成 zip 文件提交。